Topic blacklist generation using supervised learning 

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc = nlp(open(r"C:\Users\hp\Downloads\Phishing-Detection-master\new_enron.txt").read())
verb_list = []
noun_list = []
for token in doc:
    if (token.dep_ == 'nsubjpass') or (token.dep_ =='dobj'):
        x=token.head.text
        y=token.text
        verb_list.append(x)
        noun_list.append(y)
        #print(token.text, token.tag_, token.head.text, token.dep_)

In [35]:
dataset = pd.read_csv('datasetfinal3.csv')
dataset.head()



,Unnamed: 0,verb,noun,label,Unnamed: 4
0,0,Suspended,Account,1,NaN
1,1,introducing,method,1,NaN
2,2,subjected,accounts,1,NaN
3,3,request,copies,1,NaN
4,4,expired,account,1,NaN


In [36]:
formatted_dataset = pd.DataFrame()
formatted_dataset['combined'] = dataset['verb'] +" "+dataset['noun']
formatted_dataset['label'] = dataset['label']
formatted_dataset.head()

,combined,label
0,Suspended Account,1
1,introducing method,1
2,subjected accounts,1
3,request copies,1
4,expired account,1


In [37]:
X_train,X_test, y_train,y_test = train_test_split(formatted_dataset['combined'],formatted_dataset['label'],
                                                    train_size = 0.7,
                                                  shuffle=True
                                                  
                                                  )

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [38]:
print(X_train.head())
print(y_train.head())

2204      reaching agreement
385               has access
214             leave choice
1969    represent difference
246                 keep you
Name: combined, dtype: object
2204    0
385     1
214     1
1969    0
246     1
Name: label, dtype: int64


In [39]:
vectorizer =CountVectorizer()
X_train_transformed = vectorizer.fit_transform(X_train)

In [40]:
pd.DataFrame(X_train_transformed.toarray(), columns=vectorizer.get_feature_names()).head()

,0ainformation,0ainstructions,0aoneself,48hrs,ability,absent,accept,acceptance,access,accessed,...,window,work,workout,writing,www,wwwebaycom,years,you,your,yourself
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [41]:
model = MultinomialNB()
%time model.fit(X_train_transformed, y_train)


Wall time: 4 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [86]:
X_test_transformed = vectorizer.transform(X_test)
y_pred_class = model.predict(X_test_transformed)
#print(y_pred_class)
prediction = model.predict_proba(X_test_transformed)
#print(prediction,X_test)


In [87]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
results = confusion_matrix(y_test,y_pred_class) 
print('Confusion Matrix :')
print(results) 
print('Accuracy Score :',accuracy_score(y_test, y_pred_class))
print('Report : ')
print(classification_report(y_test, y_pred_class))


Confusion Matrix :
[[230  85]
 [ 14 371]]
Accuracy Score : 0.8585714285714285
Report : 
             precision    recall  f1-score   support

          0       0.94      0.73      0.82       315
          1       0.81      0.96      0.88       385

avg / total       0.87      0.86      0.86       700



In [88]:
X_test_transformed = vectorizer.transform(formatted_dataset['combined'])
y_pred_class = model.predict(X_test_transformed)
#print(y_pred_class)
prediction = model.predict_proba(X_test_transformed)
#print(prediction,X_test)
X_list = list(formatted_dataset['combined'])

In [89]:
#X_list = list(X_test)
len(prediction)
#len(X_list)

2331

In [90]:
#type(prediction[0][1])
count=0
list_pair1 = []
for i in range (0,len(X_list)):
    if(prediction[i][1]>=0.8):
      count+=1
      #print(X_list[i] + "   " + str(prediction[i][1]))
      list_pair1.append(X_list[i])
      
print(count)
      

1198


In [100]:
list_pair1 = set(list_pair1)
print(len(list_pair1))
list_pair2 = list(list_pair1)

339


In [101]:
import pandas as pd
df = pd.DataFrame(list_pair2,columns=['blacklist'])
df.to_csv('new_blacklist.csv')

In [17]:
print(prediction)

[[7.96353432e-04 9.99203647e-01]
 [6.27463235e-02 9.37253676e-01]
 [6.66506064e-02 9.33349394e-01]
 ...
 [8.65354541e-01 1.34645459e-01]
 [8.28184130e-01 1.71815870e-01]
 [9.41408357e-01 5.85916429e-02]]


# Testing 

In [102]:
import pandas as pd
df = pd.read_csv("new_blacklist.csv")

In [103]:
xLower = df["blacklist"].map(lambda x: x.lower())

In [104]:
list1=xLower.tolist()

In [105]:
list1[0:15]

['detected error',
 'initiated requests',
 'perform banking',
 'protect yourself',
 'visit policy',
 'reveive icon',
 'protecting concern',
 'activate powerseller',
 'agreed number',
 'compromised account',
 'confirm account',
 'featuring networking',
 'protecting security',
 'verify payment',
 'update recordson']

In [106]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc = nlp(open(r"C:\Users\hp\Downloads\0_false.txt").read())
_list = []

for token in doc:
    if (token.dep_ == 'nsubjpass') or (token.dep_ =='dobj'):
        x=token.head.text + " " +token.text
        _list.append(x)
        
        #print(token.text, token.tag_, token.head.text, token.dep_)

In [107]:
[x.lower() for x in _list]

['valued member',
 'confirm information',
 'completed this',
 'forced we',
 'suspend account',
 'used it',
 'thank you',
 'appreciate support',
 'keep ebay',
 'keep place',
 'thank you',
 'constitutes acceptance']

In [108]:
check =  any(item in list1 for item in _list)

In [109]:
def intersection(lst1, lst2): 
    lst3 = [value for value in list1 if value in _list] 
    return lst3 
  
print(intersection(list1,_list)) 


['valued member', 'forced we', 'keep place', 'thank you', 'confirm information', 'suspend account', 'constitutes acceptance', 'thank you']


In [110]:
check

True